In [21]:
import pandas as pd    # ура скачаем все что можно
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests
import re
import plotly.express as px
from geopy.geocoders import Nominatim
from time import sleep
from tqdm import tqdm  # Прогресс-бар
from geopy.exc import GeocoderTimedOut

In [23]:
# Спарсим кофеманию (адреса и метро)

url = "https://stars-coffee.ru/"  # Укажи правильный URL
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# Найдем все блоки с информацией о ресторанах
restaurants = soup.find_all("div", class_="t390__descr t-descr t-descr_xs")

restaurants_clean = []

for rest in restaurants:
    address_text = rest.text.strip() if rest else "Нет адреса"
    restaurants_clean.append(address_text)

restaurants_clean # ахахха спарсили просто все что можно было, попробуем теперь регулярки


['EVERY DAY BLENDРегионЛатинская Америка: Бразилия, ПатросиньоМетод обработкиНатуральныйКислотностьНизкаяТактильностьПлотная, обволакивающаяВкусВаниль, темный шоколад 75%, жареный фундукОбжаркаТемнаяАроматГрильяж и темный изюм☕PARTY BLENDРегионЛатинская Америка: Бразилия, ПатросиньоМетод обработкиНатуральныйКислотностьСредняяТактильностьСредняя, шелковистая ВкусГрецкий орех, темный шоколад, грейпфрут ОбжаркаСветлаяАроматСладкий, фруктовый☕БРАЗИЛИЯ СЕРРАДОРегионЛатинская Америка: Бразилия, Серрадо Метод обработкиНатуральныйКислотностьCредняя ТактильностьСредняя, густая и насыщеннаяВкусПерсик, ликёр, молочный шоколад ОбжаркаСветлаяАроматОрехи в сахаре, шоколад',
 'DECAFРегионЛатинская Америка: Гватемала, Уэуэтенанго Метод обработкиНатуральныйКислотностьНизкаяТактильностьСредняя, бархатистая ВкусМолочный шоколад, сдоба ОбжаркаТемнаяАроматВыпечка, сливочное масло, ваниль',
 'ЭФИОПИЯ ДАНЧЕРегионАфрика: Эфиопия, Гедео Метод обработкиМытыйКислотностьСредняя, чуть выше среднего ТактильностьСре

In [47]:
data = soup.find_all("div", class_="t390__descr t-descr t-descr_xs")


#address_pattern = r'(\w+\.?\s?\w+[\w\s\-]+,\s?\d+[^\d])' еще исключим "доб."
address_pattern = r'(\d{1,2}\s?[а-яА-Яa-zA-Z0-9\s,.\-]+(?:[^\d\s]|\s?)[^доб.\n]+)' # Честно, это написано не мной...просто надо было что-то сделать с тем, что выше

addresses = []

for item in data:
    text = item.get_text(separator="\n", strip=True)
    found_addresses = re.findall(address_pattern, text)
    addresses.extend(found_addresses)


addresses = [address for address in addresses if "доб." not in address]
addresses = [address[:-16] for address in addresses]
addresses = [address.rstrip("\n8 ") for address in addresses]
addresses = [address.split('\n', 1)[-1] if '\n' in address else address for address in addresses]
addresses = [address for address in addresses if address.strip() != '']


addresses

['11, стр. 1',
 'ТЦ Щука, Щукинская ул., 42',
 'Ходынский бульвар, 4, ТЦ АвиаПарк, 2 этаж\nАрбат 19, стр. 1',
 'Бутырский Вал ул., 10',
 'ул. Гашека, 6',
 'Ленинградский проспект, 39 стр 79, БЦ Скай Лайт',
 'Новослободская ул., 4',
 'Лазоревый проезд 1а корп 2',
 'Проезд Аэропорта, д.',
 'Ул Тверская 8 к 1 с 5',
 'Ходынский бульвар д.3\nВильгельма Пика д.11\nАвтозаводская ул., 1',
 'Кутузовский пр',
 '1',
 'Пресненская набережная 2, 1 этаж',
 'Киевское шоссе 23-й км, 1',
 'ул. Покрышкина, 4',
 'Марксистская ул., 3',
 'Кутузовский проспект, 57',
 'проспект Вернадского, 6',
 'Чистопрудный бульвар, 1А',
 'Таганская улица, 1с1',
 'М',
 '1335',
 'Кулакова 20',
 '62',
 'Комсомольский проспект д.28, МДМ',
 'Ленинский пр-т., 109',
 'Каширское ш., 61',
 'ул. Ленинская Слобода, 26',
 'Мичуринский пр., 27',
 'Москва, ул. Проспект мира, 40',
 'Садовая-Самот',
 'ул. Долгоруковская 7',
 'Шереметьевская ул., 20',
 'ВТБ Арена –',
 '36, 1 уровень',
 'г. Москва, Правды ул., 26',
 'г. Москва, ул. Арбат 3

In [51]:

cut_string = 'Ш. Шереметьевское, вл39с1 терминал В\nПроспект Вернадского 29\nМО, городской округ Красногорск, территория автодорога Балтия, 23-й километр, дом 2\nЛенинградский' # тут уже питер пошел

cut_index = next((i for i, address in enumerate(addresses) if address.startswith(cut_string)), None)

if cut_index is not None:
    addresses = addresses[:cut_index]

addresses = [address for address in addresses if len(address) >= 6] # аааа жестко кромсаем датасет



addresses

['11, стр. 1',
 'ТЦ Щука, Щукинская ул., 42',
 'Ходынский бульвар, 4, ТЦ АвиаПарк, 2 этаж\nАрбат 19, стр. 1',
 'Бутырский Вал ул., 10',
 'ул. Гашека, 6',
 'Ленинградский проспект, 39 стр 79, БЦ Скай Лайт',
 'Новослободская ул., 4',
 'Лазоревый проезд 1а корп 2',
 'Проезд Аэропорта, д.',
 'Ул Тверская 8 к 1 с 5',
 'Ходынский бульвар д.3\nВильгельма Пика д.11\nАвтозаводская ул., 1',
 'Кутузовский пр',
 'Пресненская набережная 2, 1 этаж',
 'Киевское шоссе 23-й км, 1',
 'ул. Покрышкина, 4',
 'Марксистская ул., 3',
 'Кутузовский проспект, 57',
 'проспект Вернадского, 6',
 'Чистопрудный бульвар, 1А',
 'Таганская улица, 1с1',
 'Кулакова 20',
 'Комсомольский проспект д.28, МДМ',
 'Ленинский пр-т., 109',
 'Каширское ш., 61',
 'ул. Ленинская Слобода, 26',
 'Мичуринский пр., 27',
 'Москва, ул. Проспект мира, 40',
 'Садовая-Самот',
 'ул. Долгоруковская 7',
 'Шереметьевская ул., 20',
 'ВТБ Арена –',
 '36, 1 уровень',
 'г. Москва, Правды ул., 26',
 'г. Москва, ул. Арбат 39-41',
 'Правобережная ул., 

In [52]:
df1 = pd.DataFrame(addresses, columns=['Адрес'])
df1['Названия'] = 'Stars Coffee'
df1 # покромасло его конечно знатно, но хотя что-то адекватное получилось (ну еще проверим, когда )

,Адрес,Названия
0,"11, стр. 1",Stars Coffee
1,"ТЦ Щука, Щукинская ул., 42",Stars Coffee
2,"Ходынский бульвар, 4, ТЦ АвиаПарк, 2 этаж\nАрб...",Stars Coffee
3,"Бутырский Вал ул., 10",Stars Coffee
4,"ул. Гашека, 6",Stars Coffee
5,"Ленинградский проспект, 39 стр 79, БЦ Скай Лайт",Stars Coffee
6,"Новослободская ул., 4",Stars Coffee
7,Лазоревый проезд 1а корп 2,Stars Coffee
8,"Проезд Аэропорта, д.",Stars Coffee
9,Ул Тверская 8 к 1 с 5,Stars Coffee


In [53]:
geolocator = Nominatim(user_agent="coffee_map")


df1["Latitude"] = None
df1["Longitude"] = None

for i, row in df1.iterrows():
    try:
        location = geolocator.geocode(row["Адрес"])
        if location:
            df1.at[i, "Latitude"] = location.latitude
            df1.at[i, "Longitude"] = location.longitude
    except Exception as e:
        print(f"Ошибка с адресом {row['Адрес']}: {e}")
    sleep(1)

Ошибка с адресом 11, стр. 1: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=11%2C+%D1%81%D1%82%D1%80.+1&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


Ошибка с адресом М.О., Мытищинский р-н, ш. Алтуфьевское, 1-й км, вл. 3, стр. 1: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=%D0%9C.%D0%9E.%2C+%D0%9C%D1%8B%D1%82%D0%B8%D1%89%D0%B8%D0%BD%D1%81%D0%BA%D0%B8%D0%B9+%D1%80-%D0%BD%2C+%D1%88.+%D0%90%D0%BB%D1%82%D1%83%D1%84%D1%8C%D0%B5%D0%B2%D1%81%D0%BA%D0%BE%D0%B5%2C+1-%D0%B9+%D0%BA%D0%BC%2C+%D0%B2%D0%BB.+3%2C+%D1%81%D1%82%D1%80.+1&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


Ошибка с адресом проспект Мира, 211к2: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=%D0%BF%D1%80%D0%BE%D1%81%D0%BF%D0%B5%D0%BA%D1%82+%D0%9C%D0%B8%D1%80%D0%B0%2C+211%D0%BA2&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


In [54]:
fig = px.scatter_mapbox(
    df1,
    lat="Latitude",
    lon="Longitude",
    text="Названия",
    zoom=10,
    center={"lat": 55.7558, "lon": 37.6173},  # Центр Москвы
    mapbox_style="carto-positron"
)

fig.show() # Нууууу...это даже что-то показывает

In [56]:
# Не бейте...на сайте меню было одной строчкой кода (как я понял это картинка, я не мог спарсить цены), поэтому загрузил картнку в гпт и попросил сделать df
data_stars = {
    "Напиток": [
        "Капучино", "Латте", "Раф Ванилла", "Мокка", "Мокка Белый Шоколад",
        "Голден Макиато", "Флэт Уайт", "Эспрессо 46 мл", "Фильтр-кофе", "Горячий Шоколад",
        "Латте Шоколадный Мусс с Миндалем", "Латте Лимонный Курд с Шоколадом",
        "Латте Сливочная Фисташка", "Бамбл Маття", "Бамбл Кофе", "Эспрессо-тоник",
        "Маття-тоник", "Американо Пряный Крем"
    ],
    "Маленький (350 мл)": [
        320, 365, 370, 420, 395, 405, 390, 230, 255, 370,
        410, 410, 410, 410, 410, 385, 385, 335
    ],
    "Средний (450 мл)": [
        345, 385, 390, 430, 420, 430, None, None, 265, 390,
        430, 430, 430, 435, 435, 400, 400, 360
    ],
    "Большой (550 мл)": [
        365, 415, 420, 455, 445, 445, None, None, 285, 415,
        445, 445, 445, 455, 455, 430, 430, 380
    ]
}

df_stars_beverage = pd.DataFrame(data_stars)

df_stars_beverage['Средняя стоимость'] = df_stars_beverage.iloc[:, 1:].mean(axis=1, skipna=True)

display(df_stars_beverage) # норм

stars_average_price = df_stars_beverage['Средняя стоимость'].mean()
print(stars_average_price) # ~ 389руб. - средняя стоимость напитка с страс



,Напиток,Маленький (350 мл),Средний (450 мл),Большой (550 мл),Средняя стоимость
0,Капучино,320,345.0,365.0,343.333333
1,Латте,365,385.0,415.0,388.333333
2,Раф Ванилла,370,390.0,420.0,393.333333
3,Мокка,420,430.0,455.0,435.000000
4,Мокка Белый Шоколад,395,420.0,445.0,420.000000
5,Голден Макиато,405,430.0,445.0,426.666667
6,Флэт Уайт,390,NaN,NaN,390.000000
7,Эспрессо 46 мл,230,NaN,NaN,230.000000
8,Фильтр-кофе,255,265.0,285.0,268.333333
9,Горячий Шоколад,370,390.0,415.0,391.666667


389.25925925925924


In [ ]:
url = "https://www.restoclub.ru/msk/chain/kofemanija"
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

restaurants = soup.find_all("span", class_="search-place-title__address")
metro_stations = soup.find_all("li", class_="search-place-card__info-item")

restaurants_clean = []
restaurants_metro_clean = []

for metro in metro_stations:
    metro_text = metro.find("span").text.strip() if metro.find("span") else "Нет метро"
    restaurants_metro_clean.append(metro_text)

for rest in restaurants:
    address_text = rest.text.strip() if rest else "Нет адреса"
    restaurants_clean.append(address_text)

print(restaurants_clean)
print(restaurants_metro_clean)



['ул. Н. Арбат\xa019', 'пер. М. Черкасский\xa02', 'пл. Красная\xa03', 'ул. Крузенштерна\xa0д. 10, корп. 1', '8-й км от МКАД по Киевскому ш., дер. Лапшинка, влад. 8, корп. 1', 'Аэропорт Шереметьево-2, терминал Е', 'Аэропорт Домодедово\xa01 этаж', 'пр. Кутузовский\xa017', 'просп. Академика Сахарова\xa07', 'ул. Лужники\xa024, стр. 21, Дворец тенниса']
['Арбатская', 'Нет метро', 'европейскаястейки', 'Лубянка', 'Нет метро', 'европейскаястейки', 'Охотный Ряд', 'Нет метро', 'европейскаястейкидесерты', 'Авиамоторная', 'Нет метро', 'европейская', 'Нет метро', 'европейская', 'Нет метро', 'европейская', 'Нет метро', 'Нет метро', 'европейская', 'Киевская', 'Нет метро', 'европейскаястейки', 'Красные Ворота', 'Нет метро', 'европейскаядесерты\xa0и ещё 3', 'Воробьевы горы', 'Нет метро', 'европейскаядесерты\xa0и ещё 2']


In [ ]:
url = "https://skuratovcoffee.ru/msk?srsltid=AfmBOooqHoNeixm4B4M3jjkCA3fWFUHuDwBUWGDK3RpvNTQ_GwqSYPZp"
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, "html.parser")

# restaurants = soup.find_all("span", class_="restaurant-cards__address")
address_elements = soup.find_all('a', {'class': 't-card__link'})

address = []

for addr in address_elements:
    address_text = addr.text.strip() if addr else "Нет адреса"
    address.append(address_text)

print(address)
df = pd.DataFrame(address, columns=['Адрес'])
df['Названия'] = 'Skuratov'
df

['Верхняя Радищевская, 19/3 ст2', 'Кутузовский пр-т, 36 ст39', 'Калашный переулок, 5', 'Мира, 26 ст1', 'Александра Невского, 1', 'Благовещенский переулок, 1Б', 'Нижняя Красносельская, 35 стр 49', 'Рочдельская, 15 стр.1', 'Мясницкая, 13 стр.2', 'Комсомольский пр-кт, 24/1', 'Хлебозавод — Новодмитровская, 1с27', '1-я Ямского Поля, д. 1, к. 1', 'Усиевича, 12', 'г. Химки, микрорайон ИКЕА, корпус 2']


,Адрес,Названия
0,"Верхняя Радищевская, 19/3 ст2",Skuratov
1,"Кутузовский пр-т, 36 ст39",Skuratov
2,"Калашный переулок, 5",Skuratov
3,"Мира, 26 ст1",Skuratov
4,"Александра Невского, 1",Skuratov
5,"Благовещенский переулок, 1Б",Skuratov
6,"Нижняя Красносельская, 35 стр 49",Skuratov
7,"Рочдельская, 15 стр.1",Skuratov
8,"Мясницкая, 13 стр.2",Skuratov
9,"Комсомольский пр-кт, 24/1",Skuratov


In [ ]:
geolocator = Nominatim(user_agent="coffee_map")


df["Latitude"] = None
df["Longitude"] = None

for i, row in df.iterrows():
    try:
        location = geolocator.geocode(row["Адрес"])
        if location:
            df.at[i, "Latitude"] = location.latitude
            df.at[i, "Longitude"] = location.longitude
    except Exception as e:
        print(f"Ошибка с адресом {row['Адрес']}: {e}")
    sleep(1)

Ошибка с адресом Нижняя Красносельская, 35 стр 49: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=%D0%9D%D0%B8%D0%B6%D0%BD%D1%8F%D1%8F+%D0%9A%D1%80%D0%B0%D1%81%D0%BD%D0%BE%D1%81%D0%B5%D0%BB%D1%8C%D1%81%D0%BA%D0%B0%D1%8F%2C+35+%D1%81%D1%82%D1%80+49&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


Ошибка с адресом Комсомольский пр-кт, 24/1: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=%D0%9A%D0%BE%D0%BC%D1%81%D0%BE%D0%BC%D0%BE%D0%BB%D1%8C%D1%81%D0%BA%D0%B8%D0%B9+%D0%BF%D1%80-%D0%BA%D1%82%2C+24%2F1&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


Ошибка с адресом 1-я Ямского Поля, д. 1, к. 1: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=1-%D1%8F+%D0%AF%D0%BC%D1%81%D0%BA%D0%BE%D0%B3%D0%BE+%D0%9F%D0%BE%D0%BB%D1%8F%2C+%D0%B4.+1%2C+%D0%BA.+1&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


In [ ]:
fig = px.scatter_mapbox(
    df,
    lat="Latitude",
    lon="Longitude",
    text="Названия",
    zoom=10,
    center={"lat": 55.7558, "lon": 37.6173},  # Центр Москвы
    mapbox_style="carto-positron"
)

fig.show()